### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

### Observações

#### Sobre como cria tabela CSV com limite documentos por tópico

### Código

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

In [2]:
outputs = "../outputs/"

In [3]:
onlyfiles[0:4]

C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated

['AAS_mre_ag_1973.11.20_doc_I-1.txt',
 'AAS_mre_ag_1973.11.20_doc_I-10.txt',
 'AAS_mre_ag_1973.11.20_doc_I-100.txt',
 'AAS_mre_ag_1973.11.20_doc_I-101.txt']

In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
textos = []
percentil = 10500/100
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_private_key_password = input('Entre com a senha da chave privada'),
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           #passwd=input('Entre com a senha do usuário no banco de dados'),
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs")
    row = cur.fetchone()
                    
    while row is not None:
        count += 1
        row = cur.fetchone()
    
        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')
                
        text =  row[1]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)
        
        #if count == 3: break #amostra de apenas 3 linhas. No computador do Renato, não usar o break.

In [ ]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
            
stopwords = list(set(stopwords))

In [5]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in textos]

Wall time: 1 ms


In [6]:
print(len(texts[0]))

153


In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
len(corpus)

100

In [9]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

Wall time: 5.94 s


In [10]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.000*na + 0.000*rio + 0.000*co + 0.000*Ã© + 0.000*silveira + 0.000*et + 0.000*meu + 0.000*cheÃ\xado + 0.000*mm + 0.000*lfetorhÃ£rgiridÃ£ + 0.000*sxcu + 0.000*coaracy + 0.000*mms + 0.000*brasil + 0.000*moeral + 0.000*eme/4 + 0.000*iv + 0.000*ferninbez + 0.000*amamos + 0.000*moel'),
 (1,
  '0.014*Ã© + 0.011*de + 0.011*rio + 0.011*pt + 0.008*silveira + 0.008*rjo + 0.008*zczc + 0.008*241gtx + 0.008*em + 0.005*azeredo + 0.005*vieira + 0.005*na + 0.005*entregatlg + 0.005*co + 0.005*aaa + 0.005*muito + 0.005*ap + 0.005*gb + 0.005*ll + 0.005*no'),
 (2,
  '0.019*devo + 0.017*que + 0.011*de + 0.011*do + 0.011*Ã\xad + 0.008*antonio + 0.008*silveira + 0.008*na + 0.008*co + 0.006*azeredo + 0.006*como + 0.006*francisco + 0.006*ww + 0.006*da + 0.006*vieira + 0.006*rio + 0.006*dizer + 0.006*souto + 0.006*gb + 0.006*ma'),
 (3,
  '0.011*de + 0.011*rio + 0.011*silveira + 0.011*da + 0.009*et + 0.009*vieira + 0.009*Ã\xad + 0.009*www + 0.006*may + 0.006*Ã© + 0.006*lt + 0.006*horas + 0.006*antonio +

In [11]:
lda.show_topics(num_topics=-1)

[(0,
  '0.000*na + 0.000*rio + 0.000*co + 0.000*Ã© + 0.000*silveira + 0.000*et + 0.000*meu + 0.000*cheÃ\xado + 0.000*mm + 0.000*lfetorhÃ£rgiridÃ£'),
 (1,
  '0.014*Ã© + 0.011*de + 0.011*rio + 0.011*pt + 0.008*silveira + 0.008*rjo + 0.008*zczc + 0.008*241gtx + 0.008*em + 0.005*azeredo'),
 (2,
  '0.019*devo + 0.017*que + 0.011*de + 0.011*do + 0.011*Ã\xad + 0.008*antonio + 0.008*silveira + 0.008*na + 0.008*co + 0.006*azeredo'),
 (3,
  '0.011*de + 0.011*rio + 0.011*silveira + 0.011*da + 0.009*et + 0.009*vieira + 0.009*Ã\xad + 0.009*www + 0.006*may + 0.006*Ã©'),
 (4,
  "0.015*lt + 0.008*azeredo + 0.008*hl + 0.008*nte + 0.008*amigo + 0.008*hhjÃ\xad + 0.008*buenosaires/argentin + 0.008*gclgv'swesso + 0.008*dinÃ¡ + 0.008*15.334"),
 (5,
  '0.000*de + 0.000*la + 0.000*silveira + 0.000*con + 0.000*del + 0.000*brasil + 0.000*en + 0.000*Ã© + 0.000*los + 0.000*un'),
 (6,
  '0.020*Ã© + 0.015*um + 0.010*gb + 0.008*de + 0.008*ao + 0.008*silveira + 0.008*do + 0.008*no + 0.008*azeredo + 0.008*rio'),
 (7,


In [12]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[11]

(11,
 [('de', 0.00097942057144313104),
  ('trÃªs', 0.00049483476836543387),
  ('mes', 0.00046508616604505322),
  ('silveira', 0.00045289675041935361),
  ('ma', 0.00044028188041834315),
  ('antonio', 0.00043731958368705635),
  ('da', 0.00041864210993080631),
  ('azeredo', 0.00041285064769284459),
  ('oldrich', 0.00040751829396316322),
  ('du', 0.00039802368194156198),
  ('votre', 0.00038945633117758589),
  ('241gtx', 0.00038941665333581244),
  ('rjo', 0.00038735419359664205),
  ('prie', 0.00038664920313135552),
  ('le', 0.00037656519097910822),
  ('souto', 0.00037436460969822986),
  ('relations', 0.00037361348564819816),
  ('la', 0.00037128722197391357),
  ('haute', 0.0003708085405243142),
  ('entregatlg', 0.00036898617894624798)])

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

In [15]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc ORDER BY topic_id ASC, topic_score DESC")
        
        #if count == 10: break #amostra de apenas 10 linhas

········
········
[(7, 0.037450170661774709), (16, 0.095154413132696708), (17, 0.057465683604347118), (18, 0.046179084781805955), (23, 0.16766512158968519), (26, 0.041759148478535818), (27, 0.13143608125341355), (28, 0.012238000455947813), (30, 0.013940874770229945), (33, 0.015250565077622251), (36, 0.026926708626989124), (37, 0.30364027389841819), (38, 0.045450325281436728)]
rb_1974.04.17_doc_I-66 38 0.0454645586506
[(6, 0.038575519795797764), (18, 0.1063803974761902), (27, 0.28955284047592), (30, 0.25965260562263309), (33, 0.12242686972529247), (36, 0.10777620050669939), (37, 0.048135566397467328)]
onu_1974.03.12_doc_II-49 37 0.0481396695585
[(9, 0.68906249999999958), (17, 0.13375831826818013), (31, 0.11936668173181975)]
onu_1974.03.12_doc_V-2 31 0.121551285462
[(6, 0.011172074713862072), (16, 0.095361907491620129), (18, 0.050161564915850526), (27, 0.63011272917193706), (29, 0.023223684842576402), (36, 0.069142936421462686), (37, 0.10700765724139399)]
be_1974.04.16_doc_VI-18 37 0.107

#### Visualizando os tópicos com o PyLDAvis

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))